# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'06-20-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'06-20-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-06-21 04:21:41,33.93911,67.709953,98734,3934,64401.0,30399.0,Afghanistan,253.630125,3.984443
1,NaN,NaN,NaN,Albania,2021-06-21 04:21:41,41.15330,20.168300,132490,2454,129918.0,118.0,Albania,4603.864063,1.852215
2,NaN,NaN,NaN,Algeria,2021-06-21 04:21:41,28.03390,1.659600,135821,3631,94571.0,37619.0,Algeria,309.732656,2.673372
3,NaN,NaN,NaN,Andorra,2021-06-21 04:21:41,42.50630,1.521800,13842,127,13650.0,65.0,Andorra,17914.967967,0.917497
4,NaN,NaN,NaN,Angola,2021-06-21 04:21:41,-11.20270,17.873900,37678,859,31676.0,5143.0,Angola,114.640336,2.279845


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,87716,88740,89861,91458,93272,93288,96531,98734,98734,98734
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,132437,132449,132459,132461,132469,132476,132481,132484,132488,132490
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,132727,133070,133388,133742,134115,134458,134840,135219,135586,135821


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,3412,3449,3527,3612,3683,3683,3842,3934,3934,3934
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2453,2453,2453,2453,2454,2454,2454,2454,2454,2454
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3560,3565,3571,3579,3588,3598,3605,3615,3624,3631


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,60598,60939,61410,61706,61820,62397,62698,63426,63875,64401
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,129807,129826,129842,129865,129879,129888,129895,129903,129910,129918
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,92384,92620,92852,93094,93355,93586,93831,94093,94336,94571


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7215,7215,7215,7229,7230,7230,7230,7241,7241,7241
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,21774,21774,21774,21805,21833,21839,21839,21868,21868,21868
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2345,2345,2345,2345,2346,2346,2346,2345,2345,2345


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,113,113,113,113,113,113,113,113,113,113
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,313,313,313,313,313,313,313,314,314,314
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,59,59,59,59,59,59,59,59,59,59


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
655,1001.0,Autauga,Alabama,US,2021-06-21 04:21:41,32.539527,-86.644082,7241,113,NaN,NaN,"Autauga, Alabama, US",12960.675867,1.560558
692,1075.0,Lamar,Alabama,US,2021-06-21 04:21:41,33.779950,-88.096680,1468,37,NaN,NaN,"Lamar, Alabama, US",10633.828323,2.520436
693,1077.0,Lauderdale,Alabama,US,2021-06-21 04:21:41,34.901719,-87.656247,9603,250,NaN,NaN,"Lauderdale, Alabama, US",10355.983565,2.603353


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,33541887,601824,0.0
India,29881772,386708,28765738.0
Brazil,17927928,501825,15745463.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,33541887,601824,0.0,32940063.0,2021-06-21 04:21:41,37.950547,-91.419547
India,29881772,386708,28765738.0,729326.0,2021-06-21 04:21:41,23.088275,81.806127
Brazil,17927928,501825,15745463.0,1680640.0,2021-06-21 04:21:41,-12.669522,-48.480493
France,5819088,110900,402148.0,5306040.0,2021-06-21 04:21:41,6.430808,-34.730285
Turkey,5370299,49185,5232638.0,88476.0,2021-06-21 04:21:41,38.963700,35.243300


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3808258,63335,0.0
Texas,2985659,52110,0.0
Florida,2354416,37555,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3808258,63335,0.0,3744923.0,2021-06-21 04:21:41,37.843962,-120.728594
Texas,2985659,52110,0.0,2933549.0,2021-06-21 04:21:41,31.660643,-98.653069
Florida,2354416,37555,0.0,2316861.0,2021-06-21 04:21:41,28.940755,-82.700744
New York,2111845,53606,0.0,2058239.0,2021-06-21 04:21:41,42.544151,-75.474183
Illinois,1388634,25566,0.0,1363068.0,2021-06-21 04:21:41,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1247618,24454,0.0
Arizona,Maricopa,556844,10224,0.0
Illinois,Cook,556201,10461,0.0
Florida,Miami-Dade,506428,6472,0.0
Texas,Harris,403069,6548,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1247618,24454,0.0,1223164.0,2021-06-21 04:21:41,34.308284,-118.228241,6037.0
Arizona,Maricopa,556844,10224,0.0,546620.0,2021-06-21 04:21:41,33.348359,-112.491815,4013.0
Illinois,Cook,556201,10461,0.0,545740.0,2021-06-21 04:21:41,41.841448,-87.816588,17031.0
Florida,Miami-Dade,506428,6472,0.0,499956.0,2021-06-21 04:21:41,25.611236,-80.551706,12086.0
Texas,Harris,403069,6548,0.0,396521.0,2021-06-21 04:21:41,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-18,98734,132484,135219,13842,37467,1263,4242763,224086,30322,649309,...,4626915,351378,105219,4,258140,12508,312499,6875,125157,41335
2021-06-19,98734,132488,135586,13842,37604,1263,4258394,224167,30331,649445,...,4636991,353377,105610,4,259413,12978,312605,6877,126973,41628
2021-06-20,98734,132490,135821,13842,37678,1263,4268789,224227,30355,649576,...,4646068,354865,106025,4,260740,13258,312673,6885,129033,41779


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-18,3934,2454,3615,127,853,42,88247,4492,910,10677,...,128220,5207,717,1,2925,62,3548,1352,1554,1656
2021-06-19,3934,2454,3624,127,856,42,88742,4496,910,10679,...,128234,5241,718,1,2943,64,3550,1353,1595,1666
2021-06-20,3934,2454,3631,127,859,42,89043,4498,910,10680,...,128240,5271,718,1,2958,66,3551,1355,1644,1672


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-18,63426,129903,94093,13650,31569,1221,3844329,215893,23727,635544,...,15607,317016,101018,3,238241,4733,305658,3846,103884,37143
2021-06-19,63875,129910,94336,13650,31589,1221,3868105,215999,23734,635886,...,15609,320347,101311,3,239480,5054,306240,3855,105960,37167
2021-06-20,64401,129918,94571,13650,31676,1221,3890213,216049,23742,636125,...,15610,323494,101692,3,240933,5229,306240,3953,108960,37184


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,6/11/21,6/12/21,6/13/21,6/14/21,6/15/21,6/16/21,6/17/21,6/18/21,6/19/21,6/20/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,7215,7215,7215,7229,7230,7230,7230,7241,7241,7241
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1458,1458,1458,1462,1462,1465,1465,1468,1468,1468
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9582,9582,9582,9591,9592,9596,9596,9603,9603,9603
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,3121,3121,3121,3124,3125,3129,3129,3133,3133,3133
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,16234,16234,16234,16248,16259,16265,16265,16278,16278,16278


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-18,548657,70875,0,888701,345221,3806465,553200,348665,109501,49,...,866027,2983636,411071,24360,3752,678506,446682,163382,676673,61490
2021-06-19,548657,70875,0,889342,345472,3807726,553596,348665,109528,49,...,866027,2985388,411361,24360,3752,678595,447203,163382,676673,61490
2021-06-20,548657,70875,0,889727,345605,3808258,553868,348665,109548,49,...,866027,2985659,411610,24360,3752,678649,447203,163382,676673,61490


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-06-18        7241   21868    2345  2685   6945    1243   2242   14719   
2021-06-19        7241   21868    2345  2685   6945    1243   2242   14719   
2021-06-20        7241   21868    2345  2685   6945    1243   2242   14719   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-18         3728     1873  ...    2886    672     3250      802   
2021-06-19         3728     1873  ...    2886    672     3250      802   
2021-06-20         3728     1873  ...    2886    672     3250      802   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-18           4677  3797  2302          0      925    656  
2021-06-19           4677  3797  2302          0      925    656  
2021-06-20           4677  3797  2302          0      925    656  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-18,11306,373,0,17822,5874,63274,6723,8270,1679,0,...,12516,52083,2326,256,30,11342,5820,2863,8047,734
2021-06-19,11306,373,0,17838,5874,63308,6732,8270,1682,0,...,12516,52098,2328,256,30,11343,5820,2863,8047,734
2021-06-20,11306,373,0,17843,5874,63335,6731,8270,1683,0,...,12516,52110,2330,256,30,11351,5820,2863,8047,734


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-06-18         113     314      59   64    139      42     71     327   
2021-06-19         113     314      59   64    139      42     71     327   
2021-06-20         113     314      59   64    139      42     71     327   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-18          124       45  ...      30     11       31        7   
2021-06-19          124       45  ...      30     11       31        7   
2021-06-20          124       45  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-18             40    11    13          0       26      6  
2021-06-19             40    11    13          0       26      6  
2021-06-20             40    11    13          0       26      6  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-18,0.022822,0.000023,0.002811,0.000217,0.004774,0.0,0.004823,0.000384,0.000429,0.000197,...,0.002228,0.007790,0.003672,0.0,0.004975,0.029465,0.000528,0.000873,0.023829,0.009969
2021-06-19,0.000000,0.000030,0.002714,0.000000,0.003657,0.0,0.003684,0.000361,0.000297,0.000209,...,0.002178,0.005689,0.003716,0.0,0.004931,0.037576,0.000339,0.000291,0.014510,0.007088
2021-06-20,0.000000,0.000015,0.001733,0.000000,0.001968,0.0,0.002441,0.000268,0.000791,0.000202,...,0.001958,0.004211,0.003930,0.0,0.005115,0.021575,0.000218,0.001163,0.016224,0.003627


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-18,0.023946,0.0,0.002774,0.0,0.002350,0.0,0.005217,0.000223,0.0,0.000281,...,0.000086,0.010675,0.005610,0.0,0.006538,0.016393,0.000846,0.000740,0.019016,0.005464
2021-06-19,0.000000,0.0,0.002490,0.0,0.003517,0.0,0.005609,0.000890,0.0,0.000187,...,0.000109,0.006530,0.001395,0.0,0.006154,0.032258,0.000564,0.000740,0.026384,0.006039
2021-06-20,0.000000,0.0,0.001932,0.0,0.003505,0.0,0.003392,0.000445,0.0,0.000094,...,0.000047,0.005724,0.000000,0.0,0.005097,0.031250,0.000282,0.001478,0.030721,0.003601


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-18,0.011611,0.000062,0.002792,0.000733,0.014917,0.0,0.006805,0.001104,0.000295,0.000554,...,0.000385,0.011577,0.003926,0.0,0.005308,0.017193,0.001484,0.001562,0.019620,0.000916
2021-06-19,0.007079,0.000054,0.002583,0.000000,0.000634,0.0,0.006185,0.000491,0.000295,0.000538,...,0.000128,0.010507,0.002900,0.0,0.005201,0.067822,0.001904,0.002340,0.019984,0.000646
2021-06-20,0.008235,0.000062,0.002491,0.000000,0.002754,0.0,0.005715,0.000231,0.000337,0.000376,...,0.000064,0.009824,0.003761,0.0,0.006067,0.034626,0.000000,0.025422,0.028313,0.000457


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-18,0.000609,0.001484,NaN,0.000410,0.000800,0.000082,0.000858,0.000201,0.000283,0.0,...,0.00032,0.000341,0.000791,0.000123,0.001334,0.000168,0.001251,0.0,0.00018,0.001058
2021-06-19,0.000000,0.000000,NaN,0.000721,0.000727,0.000331,0.000716,0.000000,0.000247,0.0,...,0.00000,0.000587,0.000705,0.000000,0.000000,0.000131,0.001166,0.0,0.00000,0.000000
2021-06-20,0.000000,0.000000,NaN,0.000433,0.000385,0.000140,0.000491,0.000000,0.000183,0.0,...,0.00000,0.000091,0.000605,0.000000,0.000000,0.000080,0.000000,0.0,0.00000,0.000000


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-18      0.001521  0.001328 -0.000426  0.000373  0.000144  0.000805   
2021-06-19      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2021-06-20      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

Province_State                                          ... Wyoming            \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...    Park    Platte   
2021-06-18      0.000893  0.000068  0.000537  0.000534  ...     0.0 -0.001486   
2021-06-19      0.000000  0.000000  0.000000  0.000000  ...     0.0  0.000000   
2021-06-20      0.000000  0.000000  0.000000  0.000000  ...     0.0  0.000000   

Province_State                                                           \
Admin2          Sheridan Sublette Sweetwater Teton     Uinta Unassigned   
2021-06-18      0.000308      0.0   0.001285   0.0  0.000435        NaN   
2021-06-19      0.000000      0.0   0.000000   0.0  0.000000        NaN   
2021-06-20      0.000000      0.0   0.000000   0.0  0.000000        NaN   

Province_State                  
Admin2         Washakie Weston  
2021-06-18          0.0    0.0  
2021-06-19          0.0    0.0  
2021-06-20          0.0    0.0  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-18,0.001595,0.0,NaN,0.000730,0.000852,0.000285,0.001490,0.000484,0.000000,NaN,...,0.0008,0.000826,0.000861,0.0,0.0,0.000529,0.001721,0.0,0.000124,0.0
2021-06-19,0.000000,0.0,NaN,0.000898,0.000000,0.000537,0.001339,0.000000,0.001787,NaN,...,0.0000,0.000288,0.000860,0.0,0.0,0.000088,0.000000,0.0,0.000000,0.0
2021-06-20,0.000000,0.0,NaN,0.000280,0.000000,0.000426,-0.000149,0.000000,0.000595,NaN,...,0.0000,0.000230,0.000859,0.0,0.0,0.000705,0.000000,0.0,0.000000,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2021-06-18         0.0  0.003195     0.0  0.0    0.0     0.0    0.0  0.006154   
2021-06-19         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   
2021-06-20         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-06-18          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-06-19          0.0      0.0  ...     0.0    0.0      0.0      0.0   
2021-06-20          0.0      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-06-18            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-19            0.0   0.0   0.0        NaN      0.0    0.0  
2021-06-20            0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-18,0.022371,0.000034,0.002771,0.000279,0.004690,3.801793e-08,0.005312,0.000392,0.000393,0.000225,...,0.002147,0.007474,0.003506,5.782412e-19,0.005144,0.031725,0.000584,0.000606,0.024708,0.008578
2021-06-19,0.011185,0.000032,0.002743,0.000139,0.004173,1.900896e-08,0.004498,0.000377,0.000345,0.000217,...,0.002162,0.006581,0.003611,2.891206e-19,0.005038,0.034650,0.000462,0.000449,0.019609,0.007833
2021-06-20,0.005593,0.000024,0.002238,0.000070,0.003071,9.504482e-09,0.003470,0.000322,0.000568,0.000209,...,0.002060,0.005396,0.003770,1.445603e-19,0.005077,0.028113,0.000340,0.000806,0.017916,0.005730


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-18,0.025313,0.000027,0.002504,5.874786e-17,0.003948,3.265913e-11,0.005884,0.000252,3.727318e-24,0.000299,...,0.000097,0.008858,0.004046,0.0,0.006709,0.010886,0.000713,0.000554,0.021475,0.004194
2021-06-19,0.012657,0.000014,0.002497,2.937393e-17,0.003733,1.632956e-11,0.005746,0.000571,1.863659e-24,0.000243,...,0.000103,0.007694,0.002720,0.0,0.006432,0.021572,0.000638,0.000647,0.023929,0.005117
2021-06-20,0.006328,0.000007,0.002214,1.468696e-17,0.003619,8.164782e-12,0.004569,0.000508,9.318294e-25,0.000168,...,0.000075,0.006709,0.001360,0.0,0.005764,0.026411,0.000460,0.001062,0.027325,0.004359


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-06-18,0.008655,0.000070,0.002696,0.000664,0.010266,0.001442,0.006650,0.000824,0.000304,0.000544,...,0.000841,0.011112,0.003465,1.387779e-17,0.005401,0.020694,0.001387,0.005397,0.015384,0.000839
2021-06-19,0.007867,0.000062,0.002639,0.000332,0.005450,0.000721,0.006417,0.000658,0.000299,0.000541,...,0.000485,0.010810,0.003183,6.938894e-18,0.005301,0.044258,0.001645,0.003868,0.017684,0.000742
2021-06-20,0.008051,0.000062,0.002565,0.000166,0.004102,0.000360,0.006066,0.000445,0.000318,0.000458,...,0.000274,0.010317,0.003472,3.469447e-18,0.005684,0.039442,0.000823,0.014645,0.022998,0.000600


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-18,0.000409,0.000869,NaN,0.000418,0.000795,0.000171,0.000870,0.000161,0.000297,2.286937e-139,...,0.000227,0.000375,0.000815,0.000238,0.002695,0.000217,0.001144,0.000251,0.00016,0.001056
2021-06-19,0.000205,0.000434,NaN,0.000570,0.000761,0.000251,0.000793,0.000081,0.000272,1.143468e-139,...,0.000114,0.000481,0.000760,0.000119,0.001347,0.000174,0.001155,0.000126,0.00008,0.000528
2021-06-20,0.000102,0.000217,NaN,0.000501,0.000573,0.000195,0.000642,0.000040,0.000227,5.717341e-140,...,0.000057,0.000286,0.000683,0.000059,0.000674,0.000127,0.000578,0.000063,0.00004,0.000264


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-06-18      0.000834  0.000827 -0.000186  0.000459  0.000247  0.000405   
2021-06-19      0.000417  0.000413 -0.000093  0.000230  0.000123  0.000202   
2021-06-20      0.000208  0.000207 -0.000047  0.000115  0.000062  0.000101   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-06-18      0.000593  0.000125  0.000424  0.000270  ...  0.000351   
2021-06-19      0.000296  0.000062  0.000212  0.000135  ...  0.000175   
2021-06-20      0.000148  0.000031  0.000106  0.000068  ...  0.000088   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-06-18     -0.000072  0.000208  0.000048   0.001399  0.000027  0.000184   
2021-06-19     -0.000036  0.000104  0.000024   0.000700  0.000013  0.000092   
2021-06-20     -0.000018  0.000052  0.000012   0.000350  0.000007  0.000046   

Province_State                                     
Admin2         Unassigned  Washakie        Weston  
2021-06-18           -1.0  0.000066  7.517598e-07  
2021-06-19           -1.0  0.000033  3.758799e-07  
2021-06-20           -1.0  0.000017  1.879400e-07  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-06-18,0.000944,0.000042,NaN,0.000760,0.000551,0.000312,0.001189,0.000312,0.000003,NaN,...,0.000451,0.000721,0.00050,1.196805e-07,0.006543,0.000539,0.001295,0.000515,0.000577,0.000779
2021-06-19,0.000472,0.000021,NaN,0.000829,0.000275,0.000425,0.001264,0.000156,0.000895,NaN,...,0.000226,0.000505,0.00068,5.984024e-08,0.003271,0.000313,0.000648,0.000258,0.000289,0.000390
2021-06-20,0.000236,0.000011,NaN,0.000554,0.000138,0.000426,0.000558,0.000078,0.000745,NaN,...,0.000113,0.000368,0.00077,2.992012e-08,0.001636,0.000509,0.000324,0.000129,0.000144,0.000195


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State       Alabama                                        \
Admin2               Autauga   Baldwin       Barbour          Bibb   
2021-06-18      2.489482e-06  0.001601  4.143864e-09  1.808195e-15   
2021-06-19      1.244741e-06  0.000800  2.071932e-09  9.040975e-16   
2021-06-20      6.223705e-07  0.000400  1.035966e-09  4.520488e-16   

Province_State                                                                \
Admin2                Blount       Bullock        Butler   Calhoun  Chambers   
2021-06-18      2.134865e-13  7.098939e-13  1.756980e-09  0.003077  0.000985   
2021-06-19      1.067433e-13  3.549470e-13  8.784898e-10  0.001538  0.000492   
2021-06-20      5.337164e-14  1.774735e-13  4.392449e-10  0.000769  0.000246   

Province_State                ...       Wyoming                              \
Admin2              Cherokee  ...          Park        Platte      Sheridan   
2021-06-18      1.973247e-20  ...  1.114345e-28  4.523083e-45  8.416228e-31   
2021-06-19      9.866233e-21  ...  5.571724e-29  2.261541e-45  4.208114e-31   
2021-06-20      4.933116e-21  ...  2.785862e-29  1.130771e-45  2.104057e-31   

Province_State                                                              \
Admin2              Sublette Sweetwater     Teton         Uinta Unassigned   
2021-06-18      3.697325e-51   0.001603  0.000217  1.184238e-15   0.752941   
2021-06-19      1.848662e-51   0.000801  0.000109  5.921189e-16   0.752941   
2021-06-20      9.243312e-52   0.000401  0.000054  2.960595e-16   0.752941   

Province_State                          
Admin2              Washakie    Weston  
2021-06-18      2.297210e-43  0.000195  
2021-06-19      1.148605e-43  0.000098  
2021-06-20      5.743025e-44  0.000049  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,f77912d0b80d579fbb6202fa1a90554fc4dc1443,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
